## Entity Linking with Splink & Databricks

Why not ARC? [ARC](https://github.com/databricks-industry-solutions/auto-data-linkage) is a DBX wrapper on Splink used on many workloads but it's not actively maintained anymore as most customers can now use Splink directly. I was helping a partner with Splink for their identity linking requirements and figured that Splink in its current form isn't compatible with Databricks' serverless compute. I went ahead and implemented a fix for it which is working fine from this notebook attached to a serverless compute. I'm planning to contribute it back to Splink when I find time but [here](https://github.com/moj-analytical-services/splink/compare/master...ariforu:splink:master) are the changes made to work on serverless compute. Contact me at arijit.banerjee@databricks.com.

To use this package, you need to create a git folder in your workspace where this notebook is and point it to my repository (it's public) https://github.com/ariforu/splink.git

Then load it using

```python
sys.path.insert(0, '/Workspace/Users/<your location>/splink')
```

In [0]:
import sys, os
# Point to the parent directory that contains the "splink" package folder
user_email = spark.sql("SELECT current_user() AS email").first()["email"]
sys.path.insert(0, f"/Workspace/Users/{user_email}/splink")


Since this is a test - we're going to load a UC table with 1000 sample records in it then break it into two datasets to compare. In a real-world scenario, you'd have two datasets that you want to compare.

In [0]:
duplicated_persons_df=spark.sql("select * from arijit.dedup.duplicated_persons")
duplicated_persons_l_df, duplicated_persons_r_df = duplicated_persons_df.randomSplit([0.5, 0.5], seed=42)
print("the counts for l & r are: "+str(duplicated_persons_l_df.count())+" & "+str(duplicated_persons_r_df.count())+" respectively")

the counts for l & r are: 510 & 490 respectively


- Folowing sets up Splink to link two datasets and find matching people.
- Defines how to compare fields like first name, surname, date of birth, city, and email (using fuzzy logic where helpful).
- Connects Splink to Databricks Spark for scalable processing.
- Finally, it creates a Linker object that will run the record-linkage process using those settings and the two input tables.

In [0]:


import splink.comparison_library as cl
from splink import Linker, SettingsCreator, block_on, DatabricksAPI

settings = SettingsCreator(
    link_type="link_only",
    
    comparisons=[
        cl.NameComparison(
            "first_name",
        ),
        cl.NameComparison("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
            invalid_dates_as_null=True,
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
)
#Pass appropriate values for the following parameters. splink_uc_* make the locations where splink stores the intermediate checkpoint data. Uses splink_uc_volume when dataframe_break_lineage_method="delta_lake_files"

# dataframe_break_lineage_method="delta_lake_table",
# splink_uc_catalog='main',
# splink_uc_database='default',
# splink_uc_volume='splink'

db_api=DatabricksAPI(spark_session=spark,dataframe_break_lineage_method="delta_lake_table", splink_uc_catalog='arijit',splink_uc_database='dedup')

linker = Linker(
    [duplicated_persons_l_df, duplicated_persons_r_df],
    settings,
    db_api=db_api,
    input_table_aliases=["table_left", "table_right"],
)

In [0]:
from splink.exploratory import completeness_chart

completeness_chart(
    [duplicated_persons_l_df, duplicated_persons_r_df],
    cols=["first_name", "surname", "dob", "city", "email"],
    db_api=db_api,
    table_names_for_chart=["Dataset left", "Dataset right"],
)

alt.LayerChart(...)

### Blocking Rules

#### TL;DR — What Blocking Rules Do

Blocking rules are simple conditions used to **limit which record pairs get compared** during record linkage. Instead of comparing every record with every other record (which is slow and expensive), blocking rules create **groups (“blocks”)** of records that share something in common - such as the same surname, postcode, or email. Only records within the same block are compared in detail.

**In short:** Blocking rules **reduce the number of comparisons**, making linkage faster and more scalable,  
while still keeping most true matches in the candidate set.

Take these two for example:

```sql
l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1
```

- First names must match **exactly**.  
- Dates of birth can differ by at most **one character** (to allow for small typos).


```python
block_on("email")
```

- Only records with the **same email address** are compared.  
- Email is highly discriminative, so this rule provides strong deterministic matches.


Together, these rules define a set of **deterministic blocking conditions** that select  
record pairs likely to represent the same entity before probabilistic matching occurs.


In [0]:
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email")
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  0.00333.
This means that amongst all possible pairwise record comparisons, one in 300.05 are expected to match.  With 249,900 total possible comparisons, we expect a total of around 832.86 matching pairs



In record linkage, **m-values** and **u-values** describe how likely a particular comparison outcome is, depending on whether two records truly match or not. Splink uses unsupervised learning - it does *not* need labeled matches or non-matches. Instead, it learns directly from your current unlabeled datasets, estimating the m-values and u-values that best explain the patterns of agreement and disagreement in your data.

**m-value (“match probability”)**
- The probability that two records **are a real match** *and* show a certain level of agreement.
- Example:  
  The probability that two records belonging to the **same person** have the same surname.

**u-value (“chance agreement”)**
- The probability that two records **are not a match** but still show that same level of agreement *by coincidence*.
- Example:  
  The probability that two **different people** happen to share the same surname.

First train for `u-values`

In [0]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e9, seed=1)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


Second train on `m-values`

In [0]:

session_dob = linker.training.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)
session_first_name = linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("first_name")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.`dob` = r.`dob`

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.421 in the m_probability of email, level `Exact match on email`
Iteration 2: Largest change in params was 0.107 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0528 in the m_probability of surname, level `All other comparisons`
Iteration 4: Largest change in params was 0.018 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00738 in probability_two_random_records_match
Iteration 6: Largest change in params was

Splink model is ready - now predict for matches.

In [0]:

#results = linker.inference.predict(threshold_match_probability=.9)
results = linker.inference.predict(threshold_match_probability=.9999999)
display(results.as_spark_dataframe())

Blocking time: 3.59 seconds
Predict time: 18.34 seconds


match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,surname_r,gamma_surname,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email
23.34125120584795,0.9999999059013897,table_left,table_right,430,429,Archie,Archie,4,Scott,Scott,4,1974-08-21,1974-08-21,5,Liverpool,Liverpool,1,a.s@humphrey.com,a.s@humphrey.com,4
24.053241804092092,0.9999999425549384,table_left,table_right,953,952,GergGe,George,2,Davies,Davies,4,2004-06-23,2004-06-23,5,Middlesbrough,Middlesbrough,1,gdavies72@conner-rose.com,gdavies72@conner-rose.com,4
26.063717230319043,0.9999999857426336,table_left,table_right,307,308,Harley,Harley,4,Ros,Ros,4,2000-07-16,2000-07-16,5,Swindon,Swindon,1,harley.rose50@sawyer-lin.org,harley.rose50@sawyer-lin.org,4
23.578290403148795,0.9999999201587537,table_left,table_right,864,861,Lily,Lily,4,Robinson,Robinson,4,2009-07-06,2009-07-06,5,Salford,Salford,1,lily.robinson73@peterson.biz,lily.robinson73@peterson.biz,4
23.90315861964794,0.9999999362570385,table_left,table_right,300,301,Harrison,Harrosin,3,Marshall,Marshall,4,1975-09-11,1974-09-11,4,Southampton,Southampton,1,harrisonm@patrick.com,harrisonmk@ptrick.com,2
25.361539596886313,0.9999999768038539,table_left,table_right,773,776,EvEa,Eva,3,Armstrong,Armstrong,4,2017-04-23,2017-04-23,5,Peterborough,Peterborough,1,e.a.rmstrong16@odonnell.info,er.armstrong16@odonnelloinf.,2
24.51256545401235,0.9999999582186461,table_left,table_right,759,760,Henr,Henry,3,Day,Day,4,2002-09-15,2002-09-15,5,Leeds,Leeds,1,hday48@thtomas-acrter.com,hday48@thomas-carter.com,3
25.652945330131143,0.9999999810463014,table_left,table_right,578,579,Jesisia,Jessica,2,Owen,Owen,4,1974-11-17,1974-11-17,5,Walsall,Walsall,1,jessica.owen@elliott.og,jessica.owen@elliott.org,3
24.88314498467722,0.9999999676833036,table_left,table_right,696,701,Hunter,Hunter,4,Noah,Noah,4,1973-09-29,1973-09-29,5,Manchester,Manchester,1,noahh78@diaz-clark.com,noahh78@diaz-clark.com,4


Save and load model

In [0]:
linker.misc.save_model_to_json("my_trained_model.json", overwrite=True)


{'link_type': 'link_only',
 'probability_two_random_records_match': 0.003332761676099011,
 'retain_matching_columns': True,
 'retain_intermediate_calculation_columns': False,
 'additional_columns_to_retain': [],
 'sql_dialect': 'spark',
 'linker_uid': 'jl1yw7up',
 'em_convergence': 0.0001,
 'max_iterations': 25,
 'bayes_factor_column_prefix': 'bf_',
 'term_frequency_adjustment_column_prefix': 'tf_',
 'comparison_vector_value_column_prefix': 'gamma_',
 'unique_id_column_name': 'unique_id',
 'source_dataset_column_name': 'source_dataset',
 'blocking_rules_to_generate_predictions': [],
 'comparisons': [{'output_column_name': 'first_name',
   'comparison_levels': [{'sql_condition': '`first_name_l` IS NULL OR `first_name_r` IS NULL',
     'label_for_charts': 'first_name is NULL',
     'fix_m_probability': False,
     'fix_u_probability': False,
     'is_null_level': True},
    {'sql_condition': '`first_name_l` = `first_name_r`',
     'label_for_charts': 'Exact match on first_name',
     'm_

In [0]:
db_api2=DatabricksAPI(spark_session=spark)
loaded_linker = Linker(
    [duplicated_persons_l_df, duplicated_persons_r_df],
    settings="my_trained_model.json",
    db_api=db_api2,
    input_table_aliases=["table_left_1", "table_right_1"],
)

new_results = loaded_linker.inference.predict(threshold_match_probability=.9)
display(new_results.as_spark_dataframe())

Blocking time: 2.21 seconds
Predict time: 18.25 seconds


match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,surname_r,gamma_surname,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email
4.761803994169682,0.9644504518384642,table_left_1,table_right_1,94,91,Oscra,Moore,0,Moore,Oscar,0,2016-01-12,2016-01-12,5,Liverpool,Liverpool,1,omoore64@randall.com,omoore64@randall.com,4
16.722822901482076,0.9999907546218483,table_left_1,table_right_1,102,101,Alfie,Alfie,4,null,Griffiths,-1,2008-05-07,2008-05-07,5,Plymouth,Plymouth,1,a.griffiths@garner-bridges.com,a.griffiths@garner-bridges.com,4
4.330640336386204,0.9526540768681427,table_left_1,table_right_1,103,105,Alfie,Alfie,4,Griffihs,Griffiths,3,2009-04-07,2008-05-07,2,Plyoutm,Plymouth,0,null,a.griffiths@garner-bridges.com,-1
17.99914883155824,0.9999961830660252,table_left_1,table_right_1,89,88,Lexi,Lexi,4,null,null,-1,1994-09-02,1994-09-02,5,Birmingham,Birmingham,1,l.gordon34@french.com,l.gordon34cfren@h.com,2
21.81620260679124,0.9999997291876248,table_left_1,table_right_1,94,90,Oscra,Osacar,2,Moore,Moore,4,2016-01-12,2016-01-12,5,Liverpool,Liverpool,1,omoore64@randall.com,omoore64@randall.com,4
4.330640336386204,0.9526540768681427,table_left_1,table_right_1,103,101,Alfie,Alfie,4,Griffihs,Griffiths,3,2009-04-07,2008-05-07,2,Plyoutm,Plymouth,0,null,a.griffiths@garner-bridges.com,-1
6.8061981135281355,0.991143406586522,table_left_1,table_right_1,104,105,Griffiths,Alfie,0,Alfie,Griffiths,0,2008-05-07,2008-05-07,5,Plymouth,Plymouth,1,a.griffiths@garner-bridges.com,a.griffiths@garner-bridges.com,4
10.315910601103196,0.9992160995760104,table_left_1,table_right_1,109,114,Oievr,Oliver,2,null,Atkinson,-1,2009-12-21,2009-12-21,5,Londn,London,0,oliver.atkinson@moran-smith.com,oliver.atkinson@moran-smith.com,4
14.38091209465184,0.9999531301311058,table_left_1,table_right_1,89,87,Lexi,Lex,3,null,Gorodon,-1,1994-09-02,1994-10-01,3,Birmingham,Birmingham,1,l.gordon34@french.com,l.gordon34@french.com,4
6.8061981135281355,0.991143406586522,table_left_1,table_right_1,104,101,Griffiths,Alfie,0,Alfie,Griffiths,0,2008-05-07,2008-05-07,5,Plymouth,Plymouth,1,a.griffiths@garner-bridges.com,a.griffiths@garner-bridges.com,4
